## 6.3 SQLite - podstawy

Napiszmy przykładową aplikację wykorzystującą bazę `SQLite` - w tym przykładzie nie będziemy wykonywać operacji na bazie asynchronicznie. Zaimplementujemy podstawowe operacje CRUD. Aplikacja będzie wykorzystywać pojedynczą aktywność.

<table><tr><td><img src="https://media3.giphy.com/media/FIcgLsiTwUhfuSjM8X/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/04Ir1MiMp02l2U1GO6/giphy.gif" width="200" /></td></tr></table>

### **Podstawy aplikacji**

Rozpocznijmy od dodania `ViewBinding` - do pliku `build.gradle(Module)` dodaję

```kotlin
android {
    ...
    buildFeatures {
        viewBinding = true
    }
}
```

Następnie napiszmy layout głównej aktywności w postaci prostego formularza.

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    tools:context=".MainActivity">

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:gravity="center"
        android:orientation="horizontal">

        <TextView
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="0.25"
            android:text="Name"
            android:textSize="18sp"/>

        <EditText
            android:id="@+id/edit_text_name"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="0.75"
            android:hint="Enter Name" />

    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:gravity="center"
        android:orientation="horizontal">

        <TextView
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="0.25"
            android:text="Index Number"
            android:textSize="18sp"/>

        <EditText
            android:id="@+id/edit_text_index"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="0.75"
            android:hint="Enter Index Number"
            android:inputType="number"
            android:importantForAutofill="no" />

    </LinearLayout>

    <Button
        android:id="@+id/add_button"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_margin="10dp"
        android:text="ADD STUDENT"/>

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/dataBase_recycler_view"
        android:layout_width="match_parent"
        android:layout_height="match_parent"/>
</LinearLayout>
```

Będziemy wykorzystywać `RecyclerView` do wyświetlania danych z bazy, więc dodajmy layout dla pojedynczego elementu

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/linear_layout_main"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:gravity="center_vertical"
    android:orientation="horizontal"
    android:padding="10dp">

    <TextView
        android:id="@+id/text_view_id"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:textColor="@android:color/black"
        android:textSize="16sp"
        android:padding="10dp"
        tools:text="1" />

    <TextView
        android:id="@+id/text_view_name"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:textColor="@android:color/black"
        android:textSize="16sp"
        tools:text="Name" />

    <TextView
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="5dp"
        android:layout_marginEnd="5dp"
        android:textSize="18sp"
        android:textStyle="bold"
        tools:text=":" />

    <TextView
        android:id="@+id/text_view_index"
        android:layout_width="0dp"
        android:layout_height="wrap_content"
        android:layout_weight="1"
        android:textColor="@android:color/black"
        android:textSize="16sp"
        tools:text="Index" />

    <ImageView
        android:id="@+id/image_view_edit"
        android:layout_width="50dp"
        android:layout_height="50dp"
        android:contentDescription="image"
        android:foreground="?attr/selectableItemBackgroundBorderless"
        android:scaleType="center"
        android:src="@drawable/ic_action_edit_foreground" />

    <ImageView
        android:id="@+id/image_view_delete"
        android:layout_width="50dp"
        android:layout_height="50dp"
        android:contentDescription="image"
        android:foreground="?attr/selectableItemBackgroundBorderless"
        android:scaleType="center"
        android:src="@drawable/ic_action_delete_foreground" />
</LinearLayout>
```

Przy edycji będziemy wykorzystywać `Dialog`, więc również tutaj stwórzmy layout

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="300dp"
    android:layout_height="wrap_content"
    xmlns:tools="http://schemas.android.com/tools"
    android:orientation="vertical"
    android:padding="10dp"
    tools:context=".MainActivity">

    <TextView
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Update Student Data"
        android:textColor="@android:color/black"
        android:textSize="18sp"
        android:textStyle="bold" />

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:gravity="center"
        android:orientation="horizontal">

        <TextView
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="0.25"
            android:text="Name"
            android:textSize="18sp" />

        <EditText
            android:id="@+id/edit_text_name_update"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="0.75"
            android:hint="Enter Name" />
    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:gravity="center"
        android:orientation="horizontal">

        <TextView
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="0.25"
            android:text="Index"
            android:textSize="18sp" />

        <EditText
            android:id="@+id/edit_text_index_update"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="0.75"
            android:inputType="textEmailAddress"
            android:hint="Enter Index Number" />
    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="10dp"
        android:gravity="end"
        android:orientation="horizontal">

        <Button
            android:id="@+id/button_update"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:gravity="center"
            android:padding="10dp"
            android:text="UPDATE"
            android:textStyle="bold" />

        <Button
            android:id="@+id/button_cancel"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:gravity="center"
            android:padding="10dp"
            android:text="CANCEL"
            android:textStyle="bold" />

    </LinearLayout>
</LinearLayout>
```

Dodajmy również model danych

In [ ]:
public class Student {
    private int id = 0;
    private String name;
    private int index;

    public Student(String name, int index) {
        this.name = name;
        this.index = index;
    }
    
    public Student(int id, String name, int index) {
        this(name, index);
        this.id = id;
    }

    public int getId() {
        return id;
    }

    public String getName() {
        return name;
    }

    public int getIndex() {
        return index;
    }

    public void setName(String name) {
        this.name = name;
    }

    public void setIndex(int index) {
        this.index = index;
    }
}

### **DBHelper**

Przejdźmy do utworzenia bazy, zrobimy to w klasie o nazwie (standardowo) `dbHelper`, będzie ona rozszerzać klasę `SQLiteOpenHelper` - przyjmuje ona w konstruktorze szereg parametrów.
- `context`
- `name: String` - nazwę bazy
- `SQLiteDatabase.CursorFactory` - wymagana gdy wykorzystujemy podklasy klasy `Cursor` do wykonywania zapytań - w tym przykładzie nie będziemy z niego korzystać, więc podajemy `null`
- `version: Int` - wersja bazy

Podstawowe informacje o bazie będziemy przechowywać w `companion object`

In [ ]:
public class DBHelper extends SQLiteOpenHelper {

    private static int DATABASE_VERSION = 1;
    private static final String DATABASE_NAME = "studentsDBJava.db";
    private static final String TABLE_STUDENTS = "StudentTable";

    private static final String COLUMN_ID = "_id";
    private static final String COLUMN_NAME = "name";
    private static final String COLUMN_INDEX = "indexNumber";

    public DBHelper(@Nullable Context context) {
        super(context, DATABASE_NAME, null, DATABASE_VERSION);
    }

    @Override
    public void onCreate(SQLiteDatabase db) {

    }

    @Override
    public void onUpgrade(SQLiteDatabase db, int oldVersion, int newVersion) {
    }


Musimy zaimplementować dwie metody
- `onCreate` - wywoływana gdy plik o zadanej nazwie (`BADABASE_NAME`) nie istnieje
- `onUpgrade` - wywoływana gdy schemat bazy uległ zmianie

Rozpocznijmy od metody `onCreate`. W parametrze metody dostajemy dostęp do `SQLiteDatabase`, na tej zmiennej wykonujemy instrukcje wywołując metodę `execSQL`

In [ ]:
@Override
public void onCreate(SQLiteDatabase db) {
    String CREATE_STUDENTS_TABLE = "CREATE TABLE " +
            TABLE_STUDENTS +
            "(" +
            COLUMN_ID + " " +
            "INTEGER PRIMARY KEY," +
            COLUMN_NAME +
            " TEXT," +
            COLUMN_INDEX +
            " INTEGER" +
            ")";

    db.execSQL(CREATE_STUDENTS_TABLE);
}

W metodzie `onUpgrade` usuniemy bazę i wywołamy metodę `onCreate`

In [ ]:
@Override
public void onUpgrade(SQLiteDatabase db, int oldVersion, int newVersion) {
    db.execSQL("DROP TABLE IF EXISTS " + TABLE_STUDENTS);
    onCreate(db);
}

Zaimplementujmy kilka metod umożliwiających przeprowadzenie podstawowych operacji na bazie.

#### **ADD**

Zacznijmy od metody `addStudent`

In [ ]:
public void addStudent(Student student){}

W pierwszej kolejności musimy uzyskać dostęp do bazy w celu zapisu - robimy to wywołując metodę `getWritableDatabase`

In [ ]:
SQLiteDatabase db = this.getWritableDatabase();

Następnie przygotujmy dane do dodaniam w tym celu wykorzystujemy `ContentValues` - tworzy zestaw wartości dla `ContentResolver`

In [ ]:
ContentValues values = new ContentValues();
values.put(COLUMN_NAME, student.getName());
values.put(COLUMN_INDEX, student.getIndex());

Następnie wykonujemy metodę `insert` na bazie, przyjmuje ona trzy argumenty
- nazwę tabeli do której wykonana jest operacja
- `nullColumnHack` - umożliwia wstawienie pustej wartości
- `ContentValues` - dane do wstawienia

In [ ]:
db.insert(TABLE_STUDENTS, null, values);

Na koniec zamykamy dostęp do bazy wywołując metodę `close`

In [ ]:
db.close();

Pełna metoda `addStudent`

In [ ]:
public void addStudent(Student student){
    SQLiteDatabase db = this.getWritableDatabase();

    ContentValues values = new ContentValues();
    values.put(COLUMN_NAME, student.getName());
    values.put(COLUMN_INDEX, student.getIndex());

    db.insert(TABLE_STUDENTS, null, values);
    db.close();
}

#### **DELETE**

Przejdźmy do metody umożliwiającej usunięcie wpisu. Rozpoczynamy jak poprzednio od uzyskania dostępu do bazy w celu zapisu.

In [ ]:
public void deleteStudent(Student student){
    SQLiteDatabase db = this.getWritableDatabase();

Następnie wywołujemy metodę `delete` na bazie - przyjmuje ona kilka argumentów
- nazwę tabeli z której chcemy usunąć wpis
- `whereClause` - klauzula warunkowa - tutaj chcemy usunąć element o zadanym `id`, więc podajemy 
```kotlin
`COLUMN_ID + "=" + student._id`
```
- `whereArgs` - argumenty klauzuli `where`

In [ ]:
db.delete(
        TABLE_STUDENTS,
        COLUMN_ID + "=" + student.getId(),
        null);

Ostatnim krokiem jest zamknięcie dostępu do bazy

In [ ]:
db.close();

#### **UPDATE**

Metoda aktualizacji wygląda podobnie jak poprzednie. `update` przyjmuje trzy parametry
- `ContentValues`
- `whereClause`
- `whereArgs`

In [ ]:
public void updateStudent (int id, String name, int index){
    SQLiteDatabase db = this.getWritableDatabase();
    ContentValues contentValues = new ContentValues();
    contentValues.put(COLUMN_NAME, name);
    contentValues.put(COLUMN_INDEX, index);

    db.update(TABLE_STUDENTS,
            contentValues,
            COLUMN_ID + "=" + id,
            null);

    db.close();
}

#### **GET**

Pozostaje metoda zwracająca listę wszystkich elementów w bazie.

In [ ]:
public ArrayList<Student> getStudents() {}

W metodzie stworzymy mutowalną listę do której dodamy wszystkie elementy z bazy.

In [ ]:
ArrayList<Student> students = new ArrayList<>();

Następnie otworzymy bazę tylko do odczytu

In [ ]:
SQLiteDatabase db = this.getReadableDatabase();

Wynik zapytania sql zwracamy do obiektu typu `Cursor`

In [ ]:
Cursor cursor = db.rawQuery("SELECT * FROM " + TABLE_STUDENTS, null);

Metoda `rawQuery` przyjmuje dwa argumenty
- zapytanie jako `String`
- `selectionArgs` - tablica wszystkich argumentów

Następnie sprawdzamy czy `Cursor` nie jest pusty - wykonujemy to wywołując metodę `moveToFirst`. Przesuwa ona `Cursor` do pierwszego rzędu, zwraca `false` gdy `Cursor` jest pusty

In [ ]:
if (cursor.moveToFirst())

W warunku `if` odczytujemy wartości i dodajemy je na listę `students` dopóki `Cursor` posiada wartości.

In [ ]:
if (cursor.moveToFirst()) {
    do {
        students.add(new Student(
                cursor.getInt(0),
                cursor.getString(1),
                cursor.getInt(2)));
    } while (cursor.moveToNext());
}

W bazie mamy trzy kolumny (`id: Int`, `name: String`, `index: Int`), z `Cursor` wyciągamy trzy wartości (**ważna kolejność**) i używamy ich jako argumenty konstruktora klasy `Student`.

Ostatnim elementem jest zamknięcie dostępu do bazy oraz cursora

In [ ]:
db.close();
cursor.close();

Pełny kod metody

In [ ]:
public ArrayList<Student> getStudents() {
    ArrayList<Student> students = new ArrayList<>();

    SQLiteDatabase db = this.getReadableDatabase();

    Cursor cursor = db.rawQuery("SELECT * FROM " + TABLE_STUDENTS, null);

    if (cursor.moveToFirst()) {
        do {
            students.add(new Student(
                    cursor.getInt(0),
                    cursor.getString(1),
                    cursor.getInt(2)));
        } while (cursor.moveToNext());
    }

    db.close();
    cursor.close();
    return students;
}

### **Adapter**

Przejdźmy do `StudentAdapter`, w konstruktorze przekażemy `context` oraz `DBHandler`

In [ ]:
public class StudentAdapter extends RecyclerView.Adapter<StudentAdapter.ViewHolder>{}

W metodzie `onCreateViewHolder` wykorzystamy `ViewBinding`

In [ ]:
@NonNull
@Override
public ViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
    ItemRowBinding itemBinding = ItemRowBinding.inflate(
            LayoutInflater.from(parent.getContext()), parent, false
    );
    return new ViewHolder(itemBinding);
}

W metodzie `onBindViewHolder` wywołujemy funkcję `bind` klasy `ViewHolder`

In [ ]:
@Override
public void onBindViewHolder(@NonNull ViewHolder holder, int position) {
    Student item = dbHelper.getStudents().get(position);
    holder.bind(item);
}

Metoda `getItemCount` zawiera rozmiar całej bazy

In [ ]:
@Override
public int getItemCount() {
    return dbHelper.getStudents().size();
}

Przejdźmy do klasy `ViewHolder`

In [ ]:
public class ViewHolder extends RecyclerView.ViewHolder {
    private final ItemRowBinding itemBinding;
    public ViewHolder(@NonNull ItemRowBinding itemBinding) {
        super(itemBinding.getRoot());
        this.itemBinding = itemBinding;
    }

W metodzie `bind` w epirwszej kolejności ustawiamy pola `TextView`

In [ ]:
public void bind(Student item){
    itemBinding.textViewName.setText(item.getName());
    itemBinding.textViewIndex.setText(String.valueOf(item.getIndex()));
    itemBinding.textViewId.setText(String.valueOf(item.getId()));
}

Następnie obsłużmy metodę `onClick` dla `ImageView` odpowiadającego za usuwanie elementów

In [ ]:
itemBinding.imageViewDelete.setOnClickListener(v -> {
    dbHelper.deleteStudent(item);
    notifyItemRemoved(getAdapterPosition());
});

Następnie obsłużymy zdarzenie `onClick` pola `ImgaView` odpowiedzialnego za aktualizację wpisu, wywołamy w niej metodę `setupDialog` którą następnie zaimplementujemy

In [ ]:
itemBinding.imageViewEdit.setOnClickListener(v -> {
    setupDialog(item);
});

Pełna metoda metoda `bind`

In [ ]:
public void bind(Student item){
    itemBinding.textViewName.setText(item.getName());
    itemBinding.textViewIndex.setText(String.valueOf(item.getIndex()));
    itemBinding.textViewId.setText(String.valueOf(item.getId()));

    itemBinding.imageViewDelete.setOnClickListener(v -> {
        dbHelper.deleteStudent(item);
        notifyItemRemoved(getAdapterPosition());
    });

    itemBinding.imageViewEdit.setOnClickListener(v -> {
        setupDialog(item);
    });
}

Następnie do klasy `ViewHolder` dodajmy metodę tworzącą `Dialog` który posłuży do edycji. Również tutaj wykorzystamy `ViewBinding`

In [ ]:
private void setupDialog(Student item){
    Dialog dialog = new Dialog(context);
    DialogUpdateBinding dialogBinding = DialogUpdateBinding.inflate(
            LayoutInflater.from(context)
    );
    dialog.setCancelable(false);
    dialog.setContentView(dialogBinding.getRoot());

`setCancelable` ustawiony na `false` wyłączy domyślny przycisk `Cancel`. Następnie obsłużymy wszystkie elementy `Dialog`

In [ ]:
dialogBinding.editTextIndexUpdate.setText(String.valueOf(item.getIndex()));
dialogBinding.editTextNameUpdate.setText(item.getName());

dialogBinding.buttonUpdate.setOnClickListener(v -> {
    updateDialog(dialogBinding, item, dialog);
});

dialogBinding.buttonCancel.setOnClickListener(v -> {
    dialog.dismiss();
});
dialog.show();

W obsłudze zdarzenia `onClick` przycisku `Update` wywołujemy metodę `updateDialog` przyjmującą trzy argumenty. Zaimplementujmy ją - chcemy wykonać aktualizację wpisu w bazie na podstawie zawartości pól `EditText`, w pierwszej kolejności sprawdzimy czy pola nie są puste

In [ ]:
private void updateDialog(
        DialogUpdateBinding binding,
        Student item,
        Dialog dialog){
    String updateName = binding.editTextNameUpdate.getText().toString();
    String updateIndex = binding.editTextIndexUpdate.getText().toString();

    if (!updateName.isEmpty() && !updateIndex.isEmpty()){}

Następnie wykonujemy `update` z klasy `dbHandler`, powiadamiamy adapter o zmianie i wyłączamy dialog

In [ ]:
if (!updateName.isEmpty() && !updateIndex.isEmpty()){
    dbHelper.updateStudent(item.getId(), updateName, Integer.parseInt(updateIndex));
    notifyItemChanged(item.getId() - 1);
    dialog.dismiss();
}

Pełny kod klasy `ViewHolder`

In [ ]:
public class ViewHolder extends RecyclerView.ViewHolder {
    private final ItemRowBinding itemBinding;
    public ViewHolder(@NonNull ItemRowBinding itemBinding) {
        super(itemBinding.getRoot());
        this.itemBinding = itemBinding;
    }

    public void bind(Student item){
        itemBinding.textViewName.setText(item.getName());
        itemBinding.textViewIndex.setText(String.valueOf(item.getIndex()));
        itemBinding.textViewId.setText(String.valueOf(item.getId()));

        itemBinding.imageViewDelete.setOnClickListener(v -> {
            dbHelper.deleteStudent(item);
            notifyItemRemoved(item.getId() - 1);
        });

        itemBinding.imageViewEdit.setOnClickListener(v -> {
            setupDialog(item);
        });
    }

    private void setupDialog(Student item){
        Dialog dialog = new Dialog(context);
        DialogUpdateBinding dialogBinding = DialogUpdateBinding.inflate(
                LayoutInflater.from(context)
        );
        dialog.setCancelable(false);
        dialog.setContentView(dialogBinding.getRoot());

        dialogBinding.editTextIndexUpdate.setText(String.valueOf(item.getIndex()));
        dialogBinding.editTextNameUpdate.setText(item.getName());

        dialogBinding.buttonUpdate.setOnClickListener(v -> {
            updateDialog(dialogBinding, item, dialog);
        });

        dialogBinding.buttonCancel.setOnClickListener(v -> {
            dialog.dismiss();
        });
        dialog.show();
    }

    private void updateDialog(
            DialogUpdateBinding binding,
            Student item,
            Dialog dialog){
        String updateName = binding.editTextNameUpdate.getText().toString();
        String updateIndex = binding.editTextIndexUpdate.getText().toString();

        if (!updateName.isEmpty() && !updateIndex.isEmpty()){
            dbHelper.updateStudent(item.getId(), updateName, Integer.parseInt(updateIndex));
            notifyItemChanged(item.getId() - 1);
            dialog.dismiss();
        }
    }
}

### **Aktywność**

Dodajmy `ViewBinding`

In [ ]:
private ActivityMainBinding binding;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        binding = ActivityMainBinding.inflate(getLayoutInflater());
        setContentView(binding.getRoot());
    }

Zainicjujmy `DBHandler` oraz zamknijmy dostęp do bazy w metodzie `onDestroy`

In [ ]:
class MainActivity : AppCompatActivity() {
    private DBHelper dbHelper;
    ...
    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        binding = ActivityMainBinding.inflate(getLayoutInflater());
        setContentView(binding.getRoot());

        dbHelper = new DBHelper(this);
    }
    
    @Override
    protected void onDestroy() {
        dbHelper.close();
        super.onDestroy();
    }
}

Dodajmy `RecyclerView` w metodzie `onCreate`

In [ ]:
binding.dataBaseRecyclerView.setLayoutManager(
        new LinearLayoutManager(this)
);
binding.dataBaseRecyclerView.setAdapter(
        new StudentAdapter(this, dbHelper)
);

Następnie obsłużmy zdarzenie `onClick` przycisku odpowiadającego za dodawanie elementów do bazy. Wpierw sprawdzamy czy pola `EditText` nie są puste, następnie wywołujemy metodę `addStudent` klasy `DBHandler`. Na koniec czyścimy pola `EditText` oraz powiadamiamy adapter o dodaniu nowej pozycji.

In [ ]:
binding.addButton.setOnClickListener(v -> {
    String name = binding.editTextName.getText().toString();
    String index = binding.editTextIndex.getText().toString();
    if (!name.isEmpty() && !index.isEmpty()) {
        dbHelper.addStudent(new Student(name, Integer.parseInt(index)));
        binding.editTextIndex.getText().clear();
        binding.editTextName.getText().clear();
    }

    Objects.requireNonNull(
        binding.dataBaseRecyclerView.getAdapter())
            .notifyItemInserted(dbHelper.getStudents().size());
});

Możemy przetestować aplikację

<table><tr><td><img src="https://media3.giphy.com/media/FIcgLsiTwUhfuSjM8X/giphy.gif" width="150" /></td><td><img src="https://media2.giphy.com/media/04Ir1MiMp02l2U1GO6/giphy.gif" width="150" /></td></tr></table>